In [1]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import concurrent.futures
import time
import random
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from datetime import datetime

# Google Sheets authentication
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('inc42_ipo.json', scope)
client = gspread.authorize(creds)

# Read unique URLs from Google Sheets
def read_urls_from_google_sheet(sheet_name):
    sheet = client.open('Datalabs Companies, People Tracker').worksheet(sheet_name)
    data = sheet.col_values(4)  # Assuming URLs are in the first column
    unique_urls = list(set([url.strip() for url in data if url.strip()]))  # Remove duplicates and empty strings
    return unique_urls

# Append data to Google Sheets
def append_to_google_sheet(sheet_name, data):
    print(f"Appending data to Google Sheets sub-sheet: {sheet_name}")
    sheet = client.open('Datalabs Companies, People Tracker').worksheet(sheet_name)
    if all(isinstance(row, list) for row in data):
        sheet.append_rows(data, value_input_option='RAW')
        print(f"Data appended successfully to {sheet_name}.")
    else:
        print("Error: Data format is not correct. Data should be a list of lists.")

# Format date
def format_date(date_str):
    try:
        date_obj = datetime.strptime(date_str, '%Y-%m-%d').date()
        return date_obj.isoformat()
    except ValueError:
        return date_str

def extract_key_from_url(url):
    path = url.split('/organization/')[1]
    return path

# Scrape data
def scrape_data(url):
    payload = {'api_key': '660f587720e0ccc2ce1e4c5d928816d2', 'url': url}
    r = requests.get('https://api.scraperapi.com/', params=payload)
    soup = BeautifulSoup(r.text, 'html.parser')
    script_tag = soup.find('script', {'id': 'ng-state', 'type': 'application/json'})
    
    base_data_dict = {
        'Input URL': url,
        'No of Acquisitions': 0,
        'Acquisitions Present': 'no',
        'Transaction URL': '',
        'No of Fundings': 0,
        'Funding Present': 'no',
        'Funding Transaction URL': '',
        'No of Investments': 0,
        'Investments Present': 'no',
        'Investment Transaction URL': '',
        'No of Acquirers': 0,
        'Acquirers Present': 'no',
        'Acquirer Transaction URL': '',
        'lookup_date': format_date("2024-12-19")
    }
    
    if script_tag:
        json_data = script_tag.string
        json_dict = json.loads(json_data)
        key_part = extract_key_from_url(url)
        key_path = f'GET/v4/data/entities/organizations/{key_part}field_ids=%5B%22identifier%22,%22layout_id%22,%22facet_ids%22,%22title%22,%22short_description%22,%22is_locked%22%5D&layout_mode=view_v2'
        
        # Extract acquisition data
        acquisitions_list = json_dict.get('HttpState', {}).get(key_path, {}).get('data', {}).get('cards', {}).get('acquisitions_list', [])
        lookup_date = datetime.strptime("2024-12-19", "%Y-%m-%d").date()

        count_acquisitions = 0
        transaction_urls = []
        
        for acquisition in acquisitions_list:
            announced_on_date = format_date(acquisition['announced_on']['value'])
            if datetime.fromisoformat(announced_on_date).date() >= lookup_date:
                count_acquisitions += 1
                transaction_urls.append("https://www.crunchbase.com/acquisition/" + acquisition['identifier']['permalink'])
        
        if count_acquisitions > 0:
            base_data_dict['Acquisitions Present'] = 'yes'
            base_data_dict['No of Acquisitions'] = count_acquisitions
            base_data_dict['Transaction URL'] = ", ".join(transaction_urls)
        
        # Extract funding data
        fundings_list = json_dict.get('HttpState', {}).get(key_path, {}).get('data', {}).get('cards', {}).get('funding_rounds_list', [])
        
        count_fundings = 0
        funding_urls = []
        
        for funding in fundings_list:
            announced_on_date = format_date(funding.get('announced_on', ''))
            if datetime.fromisoformat(announced_on_date).date() >= lookup_date:
                count_fundings += 1
                funding_urls.append("https://www.crunchbase.com/funding_round/" + funding['identifier']['permalink'])
        
        if count_fundings > 0:
            base_data_dict['Funding Present'] = 'yes'
            base_data_dict['No of Fundings'] = count_fundings
            base_data_dict['Funding Transaction URL'] = ", ".join(funding_urls)
        
        # Extract investment data
        investments_list = json_dict.get('HttpState', {}).get(key_path, {}).get('data', {}).get('cards', {}).get('investments_list', [])
        
        count_investments = 0
        investment_urls = []
        
        for investment in investments_list:
            announced_on_date = format_date(investment.get('announced_on', ''))
            if datetime.fromisoformat(announced_on_date).date() >= lookup_date:
                count_investments += 1
                investment_urls.append("https://www.crunchbase.com/funding_round/" + investment['funding_round_identifier']['permalink'])
        
        if count_investments > 0:
            base_data_dict['Investments Present'] = 'yes'
            base_data_dict['No of Investments'] = count_investments
            base_data_dict['Investment Transaction URL'] = ", ".join(investment_urls)
        

        # Extract acquirer data
        acquirers_list = json_dict.get('HttpState', {}).get(key_path, {}).get('data', {}).get('cards', {}).get('acquired_by_fields', [])

        count_acquirers = 0
        acquirer_urls = []

        if acquirers_list:
            acquirer = acquirers_list
            announced_on_date = format_date(acquirer.get('acquisition_announced_on', {}).get('value', ''))
            if announced_on_date:
                announced_on_date = datetime.fromisoformat(announced_on_date).date()
                if announced_on_date >= lookup_date:
                    count_acquirers += 1
                    acquirer_urls.append("https://www.crunchbase.com/acquisition/" + acquirer['acquisition_identifier']['permalink'])

            if count_acquirers > 0:
                base_data_dict['Acquirers Present'] = 'yes'
                base_data_dict['No of Acquirers'] = count_acquirers
                base_data_dict['Acquirer Transaction URL'] = ", ".join(acquirer_urls)
            else:
                base_data_dict['Acquirers Present'] = 'no'
                base_data_dict['No of Acquirers'] = 0
                base_data_dict['Acquirer Transaction URL'] = ''

    return base_data_dict

def run_scraper_in_parallel(urls, batch_size=120):
    all_data = []
    tracker_batch = []
    filtered_batch = []
    max_workers = 30

    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_url = {executor.submit(scrape_data, url): url for url in urls}
        
        for future in concurrent.futures.as_completed(future_to_url):
            url = future_to_url[future]
            try:
                data = future.result()
                if data:
                    all_data.append(data)

                    # Prepare tracker data for the current URL
                    tracker_data = [
                        data['Input URL'], data['Acquisitions Present'], data['No of Acquisitions'], data['Transaction URL'],
                        data['Funding Present'], data['No of Fundings'], data['Funding Transaction URL'],
                        data['Investments Present'], data['No of Investments'], data['Investment Transaction URL'],
                        data['Acquirers Present'], data['No of Acquirers'], data['Acquirer Transaction URL'],
                        data['lookup_date']
                    ]
                    tracker_batch.append(tracker_data)

                    # Prepare filtered data if conditions are met
                    if any(data[col] > 0 for col in ['No of Acquisitions', 'No of Fundings', 'No of Investments', 'No of Acquirers']):
                        filtered_data = [
                            data['Input URL'], data['Acquisitions Present'], data['No of Acquisitions'], data['Transaction URL'],
                            data['Funding Present'], data['No of Fundings'], data['Funding Transaction URL'],
                            data['Investments Present'], data['No of Investments'], data['Investment Transaction URL'],
                            data['Acquirers Present'], data['No of Acquirers'], data['Acquirer Transaction URL'],
                            data['lookup_date']
                        ]
                        filtered_batch.append(filtered_data)

                    # After every 'batch_size' URLs, append data to sheets
                    if len(tracker_batch) >= batch_size:
                        append_to_google_sheet('Companies Tracker', tracker_batch)
                        append_to_google_sheet('Companies With New Data', filtered_batch)

                        # Reset batches for the next batch of URLs
                        tracker_batch = []
                        filtered_batch = []

            except Exception as e:
                print(f"Error scraping URL {url}: {e}")

        # After the loop, append any remaining data if there are less than 'batch_size' URLs left
        if tracker_batch:
            append_to_google_sheet('Companies Tracker', tracker_batch)
        if filtered_batch:
            append_to_google_sheet('Companies With New Data', filtered_batch)



# Main execution
sheet_name = "Companies"  # Replace with your actual sheet name containing URLs
urls = read_urls_from_google_sheet(sheet_name)
run_scraper_in_parallel(urls)


Appending data to Google Sheets sub-sheet: Companies Tracker
Data appended successfully to Companies Tracker.
Appending data to Google Sheets sub-sheet: Companies With New Data
Data appended successfully to Companies With New Data.
Appending data to Google Sheets sub-sheet: Companies Tracker
Data appended successfully to Companies Tracker.
Appending data to Google Sheets sub-sheet: Companies With New Data
Data appended successfully to Companies With New Data.
Appending data to Google Sheets sub-sheet: Companies Tracker
Data appended successfully to Companies Tracker.
Appending data to Google Sheets sub-sheet: Companies With New Data
Data appended successfully to Companies With New Data.
Appending data to Google Sheets sub-sheet: Companies Tracker
Data appended successfully to Companies Tracker.
Appending data to Google Sheets sub-sheet: Companies With New Data
Data appended successfully to Companies With New Data.
Appending data to Google Sheets sub-sheet: Companies Tracker
Data append

In [1]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import concurrent.futures
import time
import random
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from datetime import datetime

# Google Sheets authentication
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('inc42_ipo.json', scope)
client = gspread.authorize(creds)

# Read unique URLs from Google Sheets
def read_urls_from_google_sheet(sheet_name):
    sheet = client.open('Datalabs Companies, People Tracker').worksheet(sheet_name)
    data = sheet.col_values(6)  # Assuming URLs are in the first column
    unique_urls = list(set([url.strip() for url in data if url.strip()]))  # Remove duplicates and empty strings
    return unique_urls

# Append data to Google Sheets
def append_to_google_sheet(sheet_name, data):
    print(f"Appending data to Google Sheets sub-sheet: {sheet_name}")
    sheet = client.open('Datalabs Companies, People Tracker').worksheet(sheet_name)
    if all(isinstance(row, list) for row in data):
        sheet.append_rows(data, value_input_option='RAW')
        print(f"Data appended successfully to {sheet_name}.")
    else:
        print("Error: Data format is not correct. Data should be a list of lists.")

# Format date
def format_date(date_str):
    try:
        date_obj = datetime.strptime(date_str, '%Y-%m-%d').date()
        return date_obj.isoformat()
    except ValueError:
        return date_str

def extract_key_from_url(url):
    path = url.split('/person/')[1]
    return path

# Scrape data
def scrape_data(url):
    payload = {'api_key': '660f587720e0ccc2ce1e4c5d928816d2', 'url': url}
    r = requests.get('https://api.scraperapi.com/', params=payload)
    soup = BeautifulSoup(r.text, 'html.parser')
    script_tag = soup.find('script', {'id': 'ng-state', 'type': 'application/json'})
    
    base_data_dict = {
        'Input URL': url,
        'No of Investments': 0,
        'Investments Present': 'no',
        'Investment Transaction URL': '',
        'lookup_date': format_date("2024-12-19")
    }
    
    if script_tag:
        json_data = script_tag.string
        json_dict = json.loads(json_data)
        key_part = extract_key_from_url(url)
        key_path = f'GET/v4/data/entities/people/{key_part}field_ids=%5B%22identifier%22,%22layout_id%22,%22facet_ids%22,%22title%22,%22short_description%22,%22is_locked%22%5D&layout_mode=view_v2'
        lookup_date = datetime.strptime("2024-12-19", "%Y-%m-%d").date()

        # Extract investment data
        investments_list = json_dict.get('HttpState', {}).get(key_path, {}).get('data', {}).get('cards', {}).get('investments_list', [])
        
        count_investments = 0
        investment_urls = []
        
        for investment in investments_list:
            announced_on_date = format_date(investment.get('announced_on', ''))
            if datetime.fromisoformat(announced_on_date).date() >= lookup_date:
                count_investments += 1
                investment_urls.append("https://www.crunchbase.com/funding_round/" + investment['funding_round_identifier']['permalink'])
        
        if count_investments > 0:
            base_data_dict['Investments Present'] = 'yes'
            base_data_dict['No of Investments'] = count_investments
            base_data_dict['Investment Transaction URL'] = ", ".join(investment_urls)

    return base_data_dict


def run_scraper_in_parallel(urls, batch_size=120):
    all_data = []
    tracker_batch = []
    filtered_batch = []
    max_workers = 30

    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_url = {executor.submit(scrape_data, url): url for url in urls}
        
        for future in concurrent.futures.as_completed(future_to_url):
            url = future_to_url[future]
            try:
                data = future.result()
                if data:
                    all_data.append(data)

                    # Prepare tracker data for the current URL
                    tracker_data = [
                        data['Input URL'], data['Investments Present'], data['No of Investments'], data['Investment Transaction URL'], data['lookup_date']
                    ]
                    tracker_batch.append(tracker_data)

                    # Prepare filtered data if conditions are met
                    if any(data[col] > 0 for col in ['No of Investments']):
                        filtered_data = [
                        data['Input URL'], data['Investments Present'], data['No of Investments'], data['Investment Transaction URL'], data['lookup_date']

                        ]
                        filtered_batch.append(filtered_data)

                    # After every 'batch_size' URLs, append data to sheets
                    if len(tracker_batch) >= batch_size:
                        append_to_google_sheet('People Tracker', tracker_batch)
                        append_to_google_sheet('People With New Data', filtered_batch)

                        # Reset batches for the next batch of URLs
                        tracker_batch = []
                        filtered_batch = []

            except Exception as e:
                print(f"Error scraping URL {url}: {e}")

        # After the loop, append any remaining data if there are less than 'batch_size' URLs left
        if tracker_batch:
            append_to_google_sheet('People Tracker', tracker_batch)
        if filtered_batch:
            append_to_google_sheet('People With New Data', filtered_batch)



# Main execution
sheet_name = "People"  # Replace with your actual sheet name containing URLs
urls = read_urls_from_google_sheet(sheet_name)
run_scraper_in_parallel(urls)


Appending data to Google Sheets sub-sheet: People Tracker
Data appended successfully to People Tracker.
Appending data to Google Sheets sub-sheet: People With New Data
Data appended successfully to People With New Data.
Appending data to Google Sheets sub-sheet: People Tracker
Data appended successfully to People Tracker.
Appending data to Google Sheets sub-sheet: People With New Data
Data appended successfully to People With New Data.
Appending data to Google Sheets sub-sheet: People Tracker
Data appended successfully to People Tracker.
Appending data to Google Sheets sub-sheet: People With New Data
Data appended successfully to People With New Data.
Appending data to Google Sheets sub-sheet: People Tracker
Data appended successfully to People Tracker.
Appending data to Google Sheets sub-sheet: People With New Data
Data appended successfully to People With New Data.
Appending data to Google Sheets sub-sheet: People Tracker
Data appended successfully to People Tracker.
Appending data t